In [ ]:
from langchain.document_loaders import PyPDFium2Loader ,Docx2txtLoader
from pathlib import Path

data_dir = "data"
documents = []
pages = []
# 处理PDF文件
for pdf_file in Path(data_dir).glob("*.pdf"):
    print(f"Processing {pdf_file}")
    loader = PyPDFium2Loader(str(pdf_file))
    async for page in loader.alazy_load():
        pages.append(page)
    # print(pages[49].page_content)
    documents.extend(pages)

# 处理DOCX文件
for docx_file in Path(data_dir).glob("*.docx"):
    print(f"Processing {docx_file}")
    loader = Docx2txtLoader(str(docx_file))
    documents.extend(loader.load())

# documents

Processing data\C++语言程序设计 (第5版)   .pdf


e:\Anaconda3\envs\bigmodel\lib\site-packages\pypdfium2\_helpers\textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


·34 · C++语言程序设计(第5版)
a= al Oxff;
3)按位异或(^)
按位异或操作的作用是将两个操作数对应的每一个位进行异或，具体运算规则是：若
对应位相同，则该位的运算结果为0;若对应位不同，则该位的运算结果为1。
例如：计算 0x39^0x2a
0x39: 00111001
0x2a: (^)00101010
0x39^0x2a: 00010011
使用按位异或操作可以将操作为数中的若干指定位翻转。如果使某位与0异或，结果
是该位的原值。如果使某位与1异或，则结果与该位原来的值相反。
例如：要使01111010低4位翻转，可以与00001111进行异或：
01111010
giio+
4)按位取反(～)
按位取反是一个单目运算符，其作用是对一个二进制数的每一位取反。例如：
025:0000000000010101
～025:1111111111101010
5)移位
C++中有两个移位运算符，左移运算(<<)和右移运算(>>),都是二元运算符。移
位运算符左边的操作数是需要移位的数值，右边的操作数是左移或右移的位数。
左移是按照指定的位数将一个数的二进制值向左移位。左移后，低位补 0,移出的高位
舍弃。
右移是按照指定的位数将一个数的二进制值向右移位。右移后，移出的低位舍弃。如
果是无符号数则高位补0;如果是有符号数，则高位补符号位或补0,对这一问题不同的系统
可能有不同的处理方法。面向IA-32处理器的VC++ 2008和gcc4.2 采用的都是补符号位
的方法。
下面看两个例子。
①如果有 char型变量a的值为一8,则a在内存中的二进制补码值为11111000,于是
表达式 a>>2的值为一2。图2-2 说明了移位操作的过程。
1 1 1 1 1 0 0 0 舍弃 舍弃
图2-2 a>>2 的移位操作过程
② 表达式2<<1的值为4。图2-3说明了移位操作的过程。
这里应该注意，移位运算的结果是位运算表达式(a>>2和2<<1)的值，移位运算符
左边的表达式(变量 a和常量 2)值本身并不会被改变。

Processing data\qa.docx


In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 针对不同内容类型的分块策略
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n\n", ".", "。"]
)



split_docs = text_splitter.split_documents(documents)
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
# 使用 BERT 模型来生成嵌入
embedding = HuggingFaceBgeEmbeddings(
    model_name='./bge-large-zh-v1.5',  # 官方最新版本
    model_kwargs={
        'device': 'cuda',
        'trust_remote_code': True  # 需要授权代码执行
    },
    encode_kwargs={
        'normalize_embeddings': True,
        'batch_size': 128,  # 根据GPU显存调整
        'max_length': 1024  # 支持长文本处理
    }
)

db = FAISS.from_documents(split_docs, embedding)
db.save_local('./vector_store/bert')


In [25]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
# 使用 BERT 模型来生成嵌入
embedding = HuggingFaceBgeEmbeddings(
    model_name='./bge-large-zh-v1.5',  # 官方最新版本
    model_kwargs={
        'device': 'cuda',
        'trust_remote_code': True  # 需要授权代码执行
    },
    encode_kwargs={
        'normalize_embeddings': True,
        'batch_size': 128,  # 根据GPU显存调整
        'max_length': 1024  # 支持长文本处理
    }
)

db = FAISS.from_documents(split_docs, embedding)
db.save_local('./vector_store/bert')

In [1]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS

embedding = HuggingFaceBgeEmbeddings(
    model_name='./bge-large-zh-v1.5',
    model_kwargs={
        'device': 'cuda',

    },
    encode_kwargs={
        'normalize_embeddings': True,
        'batch_size': 512,      # 增大批处理量
        'use_fp16': True        # FP16加速推理
    }
)
db1 = FAISS.load_local(
    './vector_store/bert',
    embeddings=embedding,
    allow_dangerous_deserialization=True, 
)


C:\Users\20860\AppData\Local\Temp\ipykernel_1920\3994104198.py:4: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceBgeEmbeddings(


In [2]:
test_embedding = embedding.embed_query("维度验证文本")
current_dim = len(test_embedding)
print(f"当前模型维度: {current_dim}")
if db1.index:
    index_dim = db1.index.d
    print(f"索引维度: {index_dim}")
    assert current_dim == index_dim, "维度不匹配！"

当前模型维度: 1024
索引维度: 1024


In [16]:

from langchain.prompts import ChatPromptTemplate

template = """你是C++编程助手。

使用以下检索到的上下文来给出答案。不能回答跟问题无关的内容。

问题: {question}

上下文: {context}

回答:
"""

prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='你是C++编程助手。\n\n使用以下检索到的上下文来给出答案。不能回答跟问题无关的内容。\n\n问题: {question}\n\n上下文: {context}\n\n回答:\n'), additional_kwargs={})]


In [17]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# 基础检索器（FAISS）
faiss_retriever = db1.as_retriever(
    search_type="similarity",  # 使用相似度搜索
    search_kwargs={
        "k": 5,                # 返回前5个结果

        "nprobe": 20           # 搜索的聚类数量（平衡速度与精度）
    }
)

# 精度增强：嵌入压缩过滤器
embeddings_filter = EmbeddingsFilter(
    embeddings=embedding,
)

# 最终检索器
final_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter,
    base_retriever=faiss_retriever  # 可替换为ensemble_retriever
)

In [18]:
final_retriever.get_relevant_documents("C++中const有什么用？")


[_DocumentWithState(metadata={'source': 'data\\qa.docx'}, page_content='在函数内部：如果static被用于函数内部的变量，那么它会改变该变量的生命周期，使其在程序的整个运行期间都存在，而不是在每次函数调用结束时被销毁。这意味着，这个变量的值在函数调用之间是保持的。\n\n在函数外部：如果static被用于函数外部的全局变量或函数，那么它会将这个变量或函数的链接范围限制在它被定义的文件内。换句话说，这个变量或函数不能在其他文件中被直接访问。这可以帮助减少命名冲突，而且能提供一种控制变量和函数可见性的方式。\n\n在类中：如果static被用于类的成员变量，那么该变量将会成为这个类的所有实例共享的变量，也就是说，类的每个实例都能访问到这个同样的变量。如果static被用于类的成员函数，那么这个函数可以直接通过类来调用，而不需要创建类的实例。\n\n7. C++中，#define和const有什么区别？\n\n#define和const都可以用来定义常量，但它们在实现方式和使用上有一些区别。', state={'embedded_doc': [-0.05050833523273468, -0.007735137827694416, 0.02862340770661831, -0.016523519530892372, -0.02179504558444023, 0.02609320357441902, -0.009100784547626972, 0.050168342888355255, -0.01945657841861248, 0.02379770763218403, -0.03342280164361, -0.03695297986268997, 0.018132077530026436, 0.027101323008537292, 0.038980212062597275, 0.024504471570253372, 0.002008432988077402, 0.01813005469739437, 0.009869322180747986, 0.01407675351947546, 0.04060910642147064, -0.0490705780684948, 0.001597096

In [21]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI  # 或使用其他大模型
import re
# 优化后的提示模板（明确限定回答依据）
template = """你是一个严谨的C++专家，请严格根据提供的上下文回答问题。
若上下文不相关或不包含足够信息，请明确说明无法回答。禁止编造知识。

问题：{question}

相关上下文（最多3条）：
{context}

请用中文给出分步骤的详细解释，并在关键术语旁标注英文："""
prompt = ChatPromptTemplate.from_template(template)

# 大模型选择（根据硬件配置）

from langchain_community.llms import Ollama
llm = Ollama(model="deepseek-r1:1.5b", temperature=0,)

# 构建处理链
rag_chain = (
    {"context": lambda x: "\n\n".join([doc.page_content for doc in final_retriever.get_relevant_documents(x["question"])]),
     "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 使用示例
response = rag_chain.invoke({"question": "C++中const有什么用？"})
match = re.search(r'(?<=</think>).*', response,flags=re.DOTALL)
print(match.group().strip())

### C++ 中 const 有什么用？

1. **成员变量/函数的静态引用**：
   - `const` 可以用于声明和访问类的所有实例共享的成员变量或函数。例如，`void print(const void* arg);` 表示打印函数是常引用函数。
   
2. **指针类型的转换**：
   - `const` 可以将常对象转换为普通对象。例如，`const int& p = 5; int q = *p;`

3. **静态属性的访问**：
   - `const` 可以用于访问类中的静态属性或函数。例如，`void foo(const void* arg);` 表示函数是常引用函数。

4. **避免对象销毁**：
   - 使用 `const` 时，成员变量和函数不会随着程序结束而被销毁，保持其生命周期。这有助于提高效率。

### #define 和 const 的区别

1. **定义方式**：
   - `#define` 是一个预定义的常量定义语句，用于直接将一个表达式或变量赋予一个常数值。
   - `const` 是一个关键字，表示指针指向的值是常引用的。

2. **使用场景**：
   - `#define` 通常用于函数外部或类内部的静态变量或函数。例如：
     ```c
     #define INT int
     ```
   - `const` 可以用于声明和访问类的所有实例共享的成员变量或函数，也可以用于指针类型的转换。

3. **语法区别**：
   - `const` 是一个关键字，不能用作语句开头。
   - `#define` 通常放在语句之前，但可以放在任何地方。

4. **结果不同**：
   - 使用 `#define` 定义的变量或函数是常引用的。
   - 使用 `const` 时，指针指向的值是常引用的。

### 总结

- **const** 主要用于表示常量、访问静态属性和进行指针类型的转换。
- **#define** 是一个预定义常量定义语句，用于直接将常数值赋予变量或表达式。


In [10]:
import subprocess
import os
import sys
import tempfile

def compile_and_run_cpp_code(cpp_code):
    """
    将C++代码保存到临时文件，然后编译并运行
    :param cpp_code: C++代码字符串
    :return: 运行结果
    """
    # 创建临时文件保存C++代码
    with tempfile.NamedTemporaryFile(suffix=".cpp", delete=False) as temp_file:
        temp_file.write(cpp_code.encode("utf-8"))
        temp_file_path = temp_file.name

    try:
        # 编译C++代码
        output_file = tempfile.NamedTemporaryFile(suffix=".out", delete=False).name
        compile_command = ["g++", "-o", output_file, temp_file_path]
        compile_process = subprocess.run(
            compile_command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        if compile_process.returncode != 0:
            # 编译失败，返回错误信息
            os.remove(temp_file_path)
            os.remove(output_file)
            return f"编译错误:\n{compile_process.stderr}"

        # 运行编译后的程序
        run_process = subprocess.run(
            [output_file],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        # 删除临时文件
        os.remove(temp_file_path)
        os.remove(output_file)

        # 返回运行结果
        if run_process.returncode != 0:
            return f"运行错误:\n{run_process.stderr}"
        else:
            return f"程序输出:\n{run_process.stdout}"

    except Exception as e:
        # 删除临时文件（如果存在）
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)
        if os.path.exists(output_file):
            os.remove(output_file)
        return f"发生错误: {str(e)}"

if __name__ == "__main__":
    print("请输入C++代码（输入':exit'结束输入并编译运行）：")

    cpp_code = ""
    while True:
        line = input()
        if line == ":exit":
            break
        cpp_code += line + "\n"

    if not cpp_code:
        print("没有输入C++代码！")
        sys.exit(1)

    result = compile_and_run_cpp_code(cpp_code)
    print(result)

请输入C++代码（输入':exit'结束输入并编译运行）：
